In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))


import DifferentiationInterface.src.diffipy.diffipy as dp

2024-06-16 19:29:25.831411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 19:29:25.831438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 19:29:25.831938: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-16 19:29:26.232065: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
backend_array = ['numpy', 'torch',  'tensorflow', 'jax', 'aadc'] # aadc not pre-computing tape yet...

for backend in backend_array:
    dp.setBackend(backend)
    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0
    r_value = 0.05
    sigma_value = 0.3
    dt_value = 1.0
    z_value = 0.5
    N = 1000000
    seed = 1

    # Define variables
    s0 = dp.variable(s0_value, 'input','s0')
    K = dp.variable(K_value, 'input','K')
    r = dp.variable(r_value, 'input','r')
    sigma = dp.variable(sigma_value, 'input','sigma')
    dt = dp.variable(dt_value, 'input','dt')
    z = dp.variable(z_value, 'randomVariableNormal','z')

    func_input_variables = [s0, K, r, sigma, dt, z]
    diff_variables = [s0, K, r, sigma, dt]

    # Record Tape: Standard Monte Carlo

    s = s0 * dp.exp((r - sigma **2 / 2) * dt + sigma * dp.sqrt(dt) * z)
    payoff = dp.if_(s > K, s - K, 0)
    PV_standard = dp.exp(-r * dt) * dp.sum(payoff) / N

    ### Test performance # seed and NewSample not working for aadc yet
    #dp.seed(seed)
    pre_computed_random_variables = z.NewSample(N) # pre-evaluate random samples

    PV_standard.run_performance_test(func_input_variables, diff_variables, warmup_iterations = 3, test_iterations = 5)


Backend              Eval-Result  mean runtime    variance runtime    gradient directions: {'s0', 'K', 'r', 'dt', 'sigma'}
numpy                10.039795    0.075103        0.000000            [0.500443682405205, -0.3636766294334847, 40.005448358471085, 39.96667528429754, 7.995221454493161]
numpy_as_func        10.039795    0.076384        0.000000            {'s0': 0.5004436804512125, 'K': -0.36367661788716527, 'r': 40.00544835758291, 'sigma': 39.966675321778666, 'dt': 7.995221461420953}
torch                10.038340    0.006090        0.000000            [0.5000073313713074, -0.3632945418357849, 39.96239471435547, 39.96961975097656, 7.993563652038574]
torch_as_func        10.038340    0.001529        0.000000            [tensor(0.5000), tensor(-0.3633), tensor(39.9624), tensor(39.9696), tensor(7.9936)]
tensorflow           9.990881     0.030842        0.000029            [0.49885055, -0.36267433, 39.894173, 39.770374, 7.9602647]
tensorflow_as_func   9.990881     0.007121        0.00

NotImplementedError: Must be implemented in subclasses